In [40]:
import warnings

from scipy.signal import resample

# import matplotlib.pyplot as plt
import neurokit2 as nk
import numpy as np
import pandas as pd
import seaborn as sns

In [41]:
warnings.simplefilter("ignore")

sns.set()

In [42]:
CLEAN_DATA_DIR = "cleaned_data"
FINAL_DATA_DIR = "data"

In [43]:
label_df = pd.read_csv(f"{CLEAN_DATA_DIR}/segments_labels.csv", index_col=0)
temp_df = pd.read_csv(f"{CLEAN_DATA_DIR}/segments_temperature.csv", index_col=0)
hr_df = pd.read_csv(f"{CLEAN_DATA_DIR}/segments_heartrate.csv", index_col=0)
gsr_df = pd.read_csv(f"{CLEAN_DATA_DIR}/segments_gsr.csv", index_col=0)
rr_df = pd.read_csv(f"{CLEAN_DATA_DIR}/segments_rr.csv", index_col=0)

# TODO remove the following
print("Done")

# check 30-second segments
print("Data shapes:")
print("Labels", label_df.shape)
print("Temperature", temp_df.shape)
print("Heartrate", hr_df.shape)
print("GSR", gsr_df.shape)
print("RR", rr_df.shape)

Done
Data shapes:
Labels (838, 21)
Temperature (838, 30)
Heartrate (838, 30)
GSR (838, 30)
RR (838, 30)


In [44]:
def extract_stat_features(df, domain=""):
    stat_features_names = [
        "mean",
        "std",
        "skew",
        "kurtosis",
        "diff",
        "diff2",
        "q25",
        "q75",
        "qdev",
        "max-min",
    ]

    final_names = [f"{domain}_{x}" for x in stat_features_names]
    values = np.column_stack(
        [
            df.mean(axis=1).values,  # mean
            df.std(axis=1).values,  # standard deviation
            df.skew(axis=1).values,  # skewness
            df.kurtosis(axis=1).values,  # kurtosis
            df.diff(axis=1).mean(axis=1).values,  # 1st derivative mean
            df.diff(axis=1).diff(axis=1).mean(axis=1).values,  # 2nd derivative mean
            df.quantile(0.25, axis=1).values,  # 25th quantile
            df.quantile(0.75, axis=1).values,  # 75th quantile
            df.quantile(0.75, axis=1).values - df.quantile(0.25, axis=1).values,
            df.max(axis=1).values - df.min(axis=1).values,
        ]
    )

    return pd.DataFrame(values, columns=final_names)


def extract_eda_features(df):
    feature_keys = [
        "SCR_Onsets",
        "SCR_Peaks",
        "SCR_Height",
        "SCR_Amplitude",
        "SCR_RiseTime",
        "SCR_Recovery",
        "SCR_RecoveryTime",
    ]

    # for each feature key we will calculate min, max and mean values
    feature_names = []
    for f in feature_keys:
        feature_names.append(f"min_{f}")
        feature_names.append(f"max_{f}")
        feature_names.append(f"mean_{f}")

    # iterate through all 30-second segments
    features_arr = []
    for i in range(len(df)):
        my_eda = df.iloc[i].dropna()
        my_eda_resampled = resample(
            my_eda.values, len(my_eda.values) * 10
        )  # upsampling (neurokit requires 10Hz sampling frequency)
        signals, info = nk.eda_process(my_eda_resampled, sampling_rate=10)

        segment_features = []
        for k in feature_keys:
            feature_min = 0
            feature_max = 0
            feature_mean = 0

            values = info[k]
            values = values[~np.isnan(values)]
            if (
                len(values) > 0
            ):  # update feature-values if there is at least 1 detected value (e.g., at least one peak), else leave 0
                feature_min = np.min(values)
                feature_max = np.max(values)
                feature_mean = np.mean(values)
            segment_features.extend([feature_min, feature_max, feature_mean])
        features_arr.append(segment_features)

    return pd.DataFrame(features_arr, columns=feature_names)


def extract_hrv_features(df):
    feature_names = [
        "HRV_RMSSD",
        "HRV_MeanNN",
        "HRV_SDNN",
        "HRV_SDSD",
        "HRV_CVNN",
        "HRV_CVSD",
        "HRV_MedianNN",
        "HRV_MadNN",
        "HRV_MCVNN",
        "HRV_IQRNN",
        "HRV_pNN50",
        "HRV_pNN20",
        "HRV_TINN",
        "HRV_HTI",
        "HRV_ULF",
        "HRV_VLF",
        "HRV_LF",
        "HRV_HF",
        "HRV_VHF",
        "HRV_LFHF",
        "HRV_LFn",
        "HRV_HFn",
        "HRV_LnHF",
        "HRV_SD1",
        "HRV_SD2",
        "HRV_SD1SD2",
        "HRV_S",
        "HRV_CSI",
        "HRV_CVI",
        "HRV_CSI_Modified",
        "HRV_PIP",
        "HRV_IALS",
        "HRV_PSS",
        "HRV_PAS",
        "HRV_GI",
        "HRV_SI",
        "HRV_AI",
        "HRV_PI",
        "HRV_C1d",
        "HRV_C1a",
        "HRV_SD1d",
        "HRV_SD1a",
        "HRV_C2d",
        "HRV_C2a",
        "HRV_SD2d",
        "HRV_SD2a",
        "HRV_Cd",
        "HRV_Ca",
        "HRV_SDNNd",
        "HRV_SDNNa",
        "HRV_ApEn",
        "HRV_SampEn",
        "HRV_MSE",
        "HRV_CMSE",
        "HRV_RCMSE",
        "HRV_DFA",
        "HRV_CorrDim",
    ]

    features_arr = []
    for i in range(len(df)):

        # noinspection PyBroadException
        try:
            rr = df.iloc[i].dropna()  # 30-second rr intervals

            # convert rr intervals to peaks array (input expected by neurokit)
            peaks_rr = np.zeros((len(rr) + 1) * 1000)
            peaks_rr[0] = 1
            prev_peak = 0
            for r in rr:
                peak_idx = prev_peak + int(r * 1000)
                prev_peak = peak_idx
                peaks_rr[peak_idx] = 1

            segment_features = nk.hrv(peaks_rr, sampling_rate=1000, show=False)
            features_arr.append(segment_features)
        except Exception:
            values = np.zeros(len(feature_names))
            segment_features = pd.DataFrame([values], columns=feature_names)
            features_arr.append(segment_features)

    return pd.concat(features_arr)

In [45]:
def compute_features() -> pd.DataFrame:  # ≈ 30 s
    temp_df_rolling_mean: pd.DataFrame = temp_df.rolling(3, axis=1).mean()
    hr_df_rolling_mean: pd.DataFrame = hr_df.rolling(3, axis=1).mean()
    gsr_df_rolling_mean: pd.DataFrame = gsr_df.rolling(3, axis=1).mean()
    rr_df_rolling_mean: pd.DataFrame = rr_df.rolling(3, axis=1).mean()

    temp_stat_features: pd.DataFrame = extract_stat_features(
        temp_df_rolling_mean, "temp"
    )
    hr_stat_features: pd.DataFrame = extract_stat_features(hr_df_rolling_mean, "hr")
    gsr_stat_features: pd.DataFrame = extract_stat_features(gsr_df_rolling_mean, "gsr")
    rr_stat_features: pd.DataFrame = extract_stat_features(rr_df_rolling_mean, "rr")

    stat_feat_all: pd.DataFrame = pd.concat(
        [temp_stat_features, hr_stat_features, gsr_stat_features, rr_stat_features],
        axis=1,
    )

    gsr_expert_features = extract_eda_features(gsr_df_rolling_mean)

    hrv_features: pd.DataFrame = extract_hrv_features(rr_df_rolling_mean)
    hrv_features.replace([np.inf, -np.inf], np.nan, inplace=True)
    good_features = hrv_features.isnull().sum() == 0
    hrv_features = hrv_features[hrv_features.columns[good_features]]
    hrv_features = hrv_features.reset_index().drop("index", axis=1)

    all_features: pd.DataFrame = pd.concat(
        [stat_feat_all, gsr_expert_features, hrv_features], axis=1
    )

    return all_features


"""print(
    "Statistical features",
    stat_feat_all.shape,
)
print("GSR expert features", gsr_expert_features.shape)
print("HRV features", hrv_features.shape)
print("Merged features", all_features.shape)
print("Nan values", all_features.isnull().sum().sum())

print(all_features.shape, label_df.shape)"""

'print(\n    "Statistical features",\n    stat_feat_all.shape,\n)\nprint("GSR expert features", gsr_expert_features.shape)\nprint("HRV features", hrv_features.shape)\nprint("Merged features", all_features.shape)\nprint("Nan values", all_features.isnull().sum().sum())\n\nprint(all_features.shape, label_df.shape)'

In [46]:
feature_df: pd.DataFrame = compute_features()

feature_df.to_csv(f"{FINAL_DATA_DIR}/features.csv")
label_df.to_csv(f"{FINAL_DATA_DIR}/labels.csv")

In [46]:
def normalize_features(features: pd.DataFrame) -> pd.DataFrame:
    # session-specific min-max normalization
    # assuming there’s one session per user
    pass

def standardize_features(features: pd.DataFrame) -> pd.DataFrame:
    # session-specific standardization
    # assuming there’s one session per user
    pass